In [82]:
import pandas as pd
from binance import Client 
import numpy as np
import time
import ta

In [83]:
api_key = 'key'
api_secret = 'secret'

In [84]:
client = Client(api_key, api_secret)
client.get_account()

{'makerCommission': 10,
 'takerCommission': 10,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'updateTime': 1642298974247,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BTC', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '0.02820324', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'EOS', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'SNT', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNT', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'GAS', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BCC', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '0.16746490', 'locked'

In [85]:
def getminutedata(symbol, interval,lookback):
    frame = pd.DataFrame(client.get_historical_klines(symbol,interval,lookback+'min ago UTC'))
    
    frame = frame.iloc[:,:6]
    frame.columns = ['Time', 'Open','High','Low','Close','Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index,unit='ms')
    frame = frame.astype(float)
    return frame

In [93]:
df = getminutedata('BTCUSDT','1m','100')
df

,Open,High,Low,Close,Volume
Time,,,,,
2022-01-16 21:24:00,42970.14,42973.23,42966.41,42968.80,3.01169
2022-01-16 21:25:00,42970.00,42970.00,42950.00,42954.83,2.16631
2022-01-16 21:26:00,42954.83,42966.61,42933.37,42963.73,8.60833
2022-01-16 21:27:00,42963.73,42970.66,42951.28,42968.10,2.75409
2022-01-16 21:28:00,42968.10,42969.08,42961.92,42965.46,2.22594
...,...,...,...,...,...
2022-01-16 22:59:00,43220.64,43220.64,43190.00,43190.01,7.92374
2022-01-16 23:00:00,43190.00,43240.36,43162.56,43206.43,41.62196
2022-01-16 23:01:00,43206.43,43228.82,43187.97,43202.26,10.80305


In [94]:
def applytechnicals(df):
    df['%K'] = ta.momentum.stoch(df.High,df.Low,df.Close, window=14, smooth_window=3)
    df['%D'] = df['%K'].rolling(3).mean()
    df['rsi'] = ta.momentum.rsi(df.Close,window=14)
    df['macd'] = ta.trend.macd_diff(df.Close)
    df.dropna(inplace=True)

In [95]:
applytechnicals(df)
df

,Open,High,Low,Close,Volume,%K,%D,rsi,macd
Time,,,,,,,,,
2022-01-16 21:57:00,42988.16,43018.31,42985.75,43007.52,16.86234,87.310361,84.721451,59.786762,4.749173
2022-01-16 21:58:00,43007.52,43018.28,42968.01,42970.79,33.72449,44.113842,67.297977,50.316728,2.220346
2022-01-16 21:59:00,42970.80,42971.07,42940.56,42951.01,4.89281,20.851464,50.758556,46.083407,-0.826811
2022-01-16 22:00:00,42951.02,43024.90,42940.00,43012.17,15.02839,86.105654,50.356987,57.882682,1.128176
2022-01-16 22:01:00,43012.18,43046.80,43012.17,43038.67,8.00622,92.838266,66.598461,61.785059,3.901434
...,...,...,...,...,...,...,...,...,...
2022-01-16 22:59:00,43220.64,43220.64,43190.00,43190.01,7.92374,62.078477,80.303403,58.340874,6.717546
2022-01-16 23:00:00,43190.00,43240.36,43162.56,43206.43,41.62196,73.401834,72.891525,61.992571,7.428987
2022-01-16 23:01:00,43206.43,43228.82,43187.97,43202.26,10.80305,70.526171,68.668827,60.541185,7.205512


In [96]:
class Signals:
    
    def __init__(self,df,lags):
        self.df = df 
        self.lags = lags
        
    def gettrigger(self):
        dfx = pd.DataFrame()
        for i in range(self.lags + 1):
            mask = (self.df['%K'].shift(i) < 20) & (self.df['%D'].shift(i) < 20)
            dfx = dfx.append(mask,ignore_index=True)
        #print(dfx.sum(axis=0)) #, dfx.sum(axis=0))
        return dfx.sum(axis = 0)
    
    def decide(self):
        self.df['trigger'] = np.where(self.gettrigger(),1,0)
        self.df['Buy'] = np.where((self.df.trigger) & 
                                 (self.df['%K'].between(20,80)) &
                                 (self.df['%D'].between(20,80)) &
                                 (self.df.rsi > 50) & (self.df.macd > 0),1,0)
        

In [99]:
inst = Signals(df,25)

In [100]:
inst.decide()

In [101]:
df[df.Buy == 1]

,Open,High,Low,Close,Volume,%K,%D,rsi,macd,trigger,Buy
Time,,,,,,,,,,,
2022-01-16 23:00:00,43190.00,43240.36,43162.56,43206.43,41.62196,73.401834,72.891525,61.992571,7.428987,1,1
2022-01-16 23:01:00,43206.43,43228.82,43187.97,43202.26,10.80305,70.526171,68.668827,60.541185,7.205512,1,1
2022-01-16 23:02:00,43202.27,43211.39,43174.09,43199.66,12.79695,68.733191,70.887065,59.604179,6.491534,1,1
2022-01-16 23:03:00,43199.65,43199.66,43181.75,43191.08,4.78908,62.816357,67.358573,56.496668,5.111122,1,1


In [ ]:
def strategy(pair, qty, open_position=False):
    df = getminutedata(pair, '1m', '100')
    inst = Signals(df, 25)
    inst.decide()
    print(f'current Close is '+str(df.CLose.iloc[-1]))
    
    if df.Buy.iloc[-1]:
        order = client.create_order(symbol=pair,
                                   side='BUY',
                                   type='MARKET',
                                   quantity=qty)
        print(order)
        buyprice = float(order['fills'][0]['price'])
        open_position = True 
    
    while open_position:
        time.sleep(0.5)
        #get the recent price by looking back 2minute
        df = getminutedata(pair,'1m','2')
        print(f'current Close' + str(df.Close.iloc[-1]))
        print(f'current Target' + str(buyprice * 1.005)
        print(f'current Stop is' + str(buyprice * 0.995))
        
        if df.Close[-1] <= buyprice * 0.995 or df.Close[-1] >= buyprice * 1.005:
            order = client.create_order(symbol = pair, side = 'SELL', type = 'MARKET', quantity = qty)
        
        print(order)
        break